In [ ]:
# !pip install datasets -qq

# !pip install transformers -qq

# !pip install accelerate -qq

# !pip install fuzzywuzzy -qq

# !pip install python-Levenshtein -qq

import re
from typing import Dict, List, Union
import random

import numpy as np
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Функция с промтом для модели

In [ ]:
def prepare_message_for_llm_v2(text: Union[str, List[str]], categories: Dict[str, list[str]]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_)
    
    system_prompt = '''Вы - эксперт по классификации новостных текстов. Задача - выбрать основную тему текста из предложенного списка.
ТРЕБОВАНИЯ:
1. Выберите ТОЛЬКО ОДНУ тему из списка
2. Ответ должен содержать ТОЛЬКО название темы, без пояснений
3. Если текст подходит под несколько тем, выберите наиболее подходящую'''

    if isinstance(text, str):
        prompt = f'''Определите основную тему текста. Выберите ОДНУ тему из списка: {categories_as_string}.
Примеры классификации:
'''
        for cur in categories_:
            for exml in categories[cur]:
                prompt += f'Текст: {exml}\nТема: {cur}\n'
        
        prompt += f'''
Текст для классификации:
Текст: {text[:1000]}
Тема:'''
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt}
        ]
    else:
        messages = []
        for it in text:
            prompt = f'''Определите основную тему текста. Выберите ОДНУ тему из списка: {categories_as_string}.
Примеры классификации:
'''
            for cur in categories_:
                for exml in categories[cur]:
                    prompt += f'Текст: {exml}\nТема: {cur}\n'
            
            prompt += f'''
Текст для классификации:
Текст: {it[:1000]}
Тема:'''
            
            messages.append([
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': prompt}
            ])
    
    return {'message_for_llm': messages}

Функция, которая выбирает примеры для промта.
Я решил выбрать поиск на основе расстояния от "центра" всей текстовой выборки.
По идее такой выбор должен привести к тому, что небольшое количество объектов будет хорошо предоставлять модели информации о конкретном типе текста.

In [ ]:
def select_representative_examples(train_set, categories, n_examples=2):
    examples_by_categories = {}
    
    for category in categories:
        category_texts = train_set.filter(lambda it: it['category'] == category)['text']
        
        if len(category_texts) == 0:
            continue
            
        vectorizer = TfidfVectorizer(max_features=1000, stop_words=None)
        try:
            tfidf_matrix = vectorizer.fit_transform(category_texts)
            centroid = tfidf_matrix.mean(axis=0)
            similarities = cosine_similarity(tfidf_matrix, centroid)
            best_indices = np.argsort(similarities.flatten())[-n_examples:][::-1]
            best_examples = [category_texts[i] for i in best_indices]
            
        except:
            best_examples = np.random.choice(category_texts, size=min(n_examples, len(category_texts)), replace=False).tolist()
        
        examples_by_categories[category] = best_examples
    
    return examples_by_categories

Функция для нормализации вывода модели
Модель хорошо поняла запрос на выдачу только категории, поэтому нет необходимости фильтровать её вывод.
Однако, модель жёстко галлюцинирует, выдавая несуществующие категории. И по сути она отвечает верно, но только по смыслу так просто нормализовать такое не выйдет, по-хорошему нужна какая-то небольшая моделька, на основе эмбеддингов которой можно будет сопоставлять схожие по смыслу слова. Но с таким подходом можно просто сразу взять нормальную модель для классификации и не воевать с промтингом.
Поэтому здесь я просто взял список галлюцинаций модели и классов из датасета и попросил составить мне словарик. Ну и фуззи файндер тоже здесь есть, куда же без него.

In [ ]:
def normalize_prediction(prediction: str, categories: List[str]) -> str:
    pred = prediction.strip().lower()
    
    if pred in categories:
        return pred
    
    synonym_map = {
        'science': 'science/technology',
        'technology': 'science/technology', 
        'tech': 'science/technology',
        'sci': 'science/technology',
        'biology': 'science/technology',
        'physics': 'science/technology',
        'chemistry': 'science/technology',
        'research': 'science/technology',
        'innovation': 'science/technology',
        'digital': 'science/technology',
        'computer': 'science/technology',
        'software': 'science/technology',
        'internet': 'science/technology',
        'telecommunications': 'science/technology',
        'telecom': 'science/technology',
        'communication': 'science/technology',
        'network': 'science/technology',
        
        # health
        'medicine': 'health',
        'medical': 'health',
        'healthcare': 'health',
        'hospital': 'health',
        'disease': 'health',
        'treatment': 'health',
        'doctor': 'health',
        'patient': 'health',
        
        # sports
        'sport': 'sports',
        'athletics': 'sports',
        'game': 'sports',
        'football': 'sports',
        'basketball': 'sports',
        'competition': 'sports',
        'championship': 'sports',
        'olympics': 'sports',
        
        # entertainment
        'entertain': 'entertainment',
        'movie': 'entertainment',
        'music': 'entertainment',
        'film': 'entertainment',
        'show': 'entertainment',
        'concert': 'entertainment',
        'festival': 'entertainment',
        'celebrity': 'entertainment',
        'arts': 'entertainment',
        'culture': 'entertainment',
        
        # politics
        'politic': 'politics',
        'government': 'politics',
        'election': 'politics',
        'political': 'politics',
        'policy': 'politics',
        'minister': 'politics',
        'president': 'politics',
        'election': 'politics',
        'religion': 'politics',
        
        # travel
        'traveling': 'travel',
        'tourism': 'travel',
        'vacation': 'travel',
        'tourist': 'travel',
        'hotel': 'travel',
        'destination': 'travel',
        'trip': 'travel',
        'journey': 'travel',
        'transportation': 'travel',
        'transport': 'travel',
        
        # geography
        'geographic': 'geography',
        'location': 'geography',
        'country': 'geography',
        'city': 'geography',
        'map': 'geography',
        'region': 'geography',
        'territory': 'geography',
        'weather': 'geography',
        'climate': 'geography',
        'nature': 'geography',
        'environment': 'geography'
    }
    
    if pred in synonym_map:
        return synonym_map[pred]
    
    for category in categories:
        if category in pred or pred in category:
            return category
    
    best_match, score = process.extractOne(pred, categories, scorer=fuzz.token_sort_ratio)
    if score > 70:
        return best_match
    
    return categories[0]

Изначально пошёл по наиболее простому пути, просто взять модельку меньшего размера, да так с ней и работал до конца лабы. Пробовал ещё пару типо оптимизированных под русский язык, но они либо работали также, либо даже хуже. Особого смысла их использовать не увидел.

Вот например такую использовал: RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4 (оказалась удивительно неточной, хотя по описанию выглядела весьма неплохой)

In [ ]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
llm_pipeline = pipeline(model=model_name, device_map='auto', torch_dtype='auto', do_sample=False, temperature=0.1, top_p=0.9, max_new_tokens=10)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Грузим датасетик...

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


Собственно применяем поиск наиболее репрезентативных примеров для промта.

In [ ]:
best_examples = select_representative_examples(train_set, list_of_categories, n_examples=3)

validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm_v2(it['text'], best_examples))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm_v2(it['text'], best_examples))

print(validation_set_for_llm)
print(validation_set['text'][0])
print(validation_set_for_llm['message_for_llm'][0])

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})
Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.
[{'content': 'Вы - эксперт по классификации новостных текстов. Задача - выбрать основную тему текста из предложенного списка.\nТРЕБОВАНИЯ:\n1. Выберите ТОЛЬКО ОДНУ тему из списка\n2. Ответ должен содержать ТОЛЬКО название темы, без пояснений\n3. Если текст подходит под несколько тем, выберите наиболее подходящую', 'role': 'system'}, {'content': 'Определите основную тему текста. Выберите ОДНУ тему из списка: entertainment, geography, health, politics, science/technology, sports, travel.\nПримеры классификации:\nТекст: Издатель игр Konami заявил сегодня в одной из японских газет, что они не будут выпускать игру Six Days in Fallujah.\nТема: entertainment\nТекст: Однако почти во всех казино, перечисленных выше, подают напитки, а некото

Делаем предикт на валидационной выборке.

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))

  0%|          | 0/99 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Нормализуем вывод и считаем метрики.

In [ ]:
y_true = validation_set['category']

classes = ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']
for pred in y_pred:
    if pred[-1]["content"] not in classes:
        print(pred)

print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

y_pred_with_normalization = [normalize_prediction(x[-1]['content'], classes) for x in y_pred]

print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

[{'content': 'Вы - эксперт по классификации новостных текстов. Задача - выбрать основную тему текста из предложенного списка.\nТРЕБОВАНИЯ:\n1. Выберите ТОЛЬКО ОДНУ тему из списка\n2. Ответ должен содержать ТОЛЬКО название темы, без пояснений\n3. Если текст подходит под несколько тем, выберите наиболее подходящую', 'role': 'system'}, {'content': 'Определите основную тему текста. Выберите ОДНУ тему из списка: entertainment, geography, health, politics, science/technology, sports, travel.\nПримеры классификации:\nТекст: Издатель игр Konami заявил сегодня в одной из японских газет, что они не будут выпускать игру Six Days in Fallujah.\nТема: entertainment\nТекст: Однако почти во всех казино, перечисленных выше, подают напитки, а некоторые из них используют и различные виды развлечений от известных брендов (в первую очередь, это крупные компании, расположенные в непосредственной близости от Альбукерке и Санта-Фе).\nТема: entertainment\nТекст: В конце 2017 года Симинофф появился на торговом 

/home/dmitry/Projects/NLP/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/dmitry/Projects/NLP/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/dmitry/Projects/NLP/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]

Повторяем для тестовой выборки.

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))


  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
y_true = test_set['category']

classes = ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']
for pred in y_pred:
    if pred[-1]["content"] not in classes:
        print(pred[-1]["content"])

print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

y_pred_with_normalization = [normalize_prediction(x[-1]['content'], classes) for x in y_pred]

print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

nature
transportation
weather
                    precision    recall  f1-score   support

     entertainment       0.82      0.74      0.78        19
         geography       0.77      1.00      0.87        17
            health       0.87      0.91      0.89        22
            nature       0.00      0.00      0.00         0
          politics       0.88      0.97      0.92        30
science/technology       0.87      0.88      0.87        51
            sports       0.86      0.76      0.81        25
    transportation       0.00      0.00      0.00         0
            travel       0.97      0.78      0.86        40
           weather       0.00      0.00      0.00         0

          accuracy                           0.86       204
         macro avg       0.60      0.60      0.60       204
      weighted avg       0.88      0.86      0.86       204

                    precision    recall  f1-score   support

     entertainment       0.82      0.74      0.78        19
      

/home/dmitry/Projects/NLP/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/dmitry/Projects/NLP/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/dmitry/Projects/NLP/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0]

### Выводы
Очень неудобно работать в таком формате, маленькие модели часто работают очень плохо, даже для таких задач, где нужно ответить одним словом. При тестах, когда не менял ничего кроме доп вывода модель спокойно могла выдовать f1 от 0.81 до 0.87, по идее параметр do_sample=False должен это убрать, но при использвании do_sample=True модель больше косячит и часто, зато способна выдать более точный ответ иногда (каким-то образом...). Голлюцинации модели отдельная интересная вещь, ведь их так просто не поправить - лучше всего с этим справятся модели, но как я писал в ячейках выше, тогда можно просто взять модель классификатор (ну или просто не промтить а брать эмбеддинги модели и на них уже обучать модель попроще).